# Предсказание оценок пользователя

<b>Рекомендательные системы</b> являются основопологающим элементов многих современных web-сервисов. Начиная от социальных систей, где нам предлагают умную ленту до интернет-магазинов, где всегда можно увидеть несколько интересных сопутствующих предложений. <b>Ценность</b> такой системы довольная очевидна $-$ мы хотим удерживать пользователя на сервисе, чтобы он приносил нам больше прибыли от просмотра рекламы или покупки новых товаров.

В этом проекте я разберу подзадачу построения рекоменадательной системы, а именно предсказание оценок пользователя для предмета, для этого изучу данные, выделю признаки и сравню несколько алгоритмов на качество.

Для начала <b>формализуем задачу</b>:

Имеется множество пользователей $U$ и множество предметов $I$. Для некоторых фильмов конкретный пользователь уже поставил оценку $r_{ui}$, надо предсказать оценку для всех остальных задач.

На основе этих данных можно предлагать пользователю товары с наибольшей вероятной оценки, но это не всегда оптимально, так как существует множество сопутствующих факторов, поэтому задача сводится к задаче ранжирования, которая решается своими способами. Именно поэтому выделяется отдельная подзадача.

Рассматривать же задачу мы будем на примере предсказывания оценок пользователя фильмам в онлайн-кинотеатре или сервисе, подобном [kinopoisk](https://www.kinopoisk.ru/).

Данные мы возьмём от организации [GroupLens](https://grouplens.org/), которая собрала оценки $138493$ пользователей по $27278$ фильмам в период с $\textit{09 января 1995}$ по $\textit{17 октября 2016}$. В общей сложности было поставлено $20000263$ оценок и $465564$ тегов. Основными источниками данных послужили сервисы [imdb](https://www.imdb.com/) и [tmdb](https://www.themoviedb.org/?language=ru).

Если у вас UNIX система и установлен kaggle API, то для скачивания данных выполните следующий блок. Так же можно скачать данные напрямую со соответствующей страницы на [kaggle](https://www.kaggle.com/grouplens/movielens-20m-dataset).

In [ ]:
!kaggle datasets download -d grouplens/movielens-20m-dataset -p ./data/

In [ ]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

%pylab inline

PATH_TO_DATA = './data/'

### Обзор данных

In [ ]:
genome_scores = pd.read_csv(PATH_TO_DATA + 'genome_scores.csv', dtype={
    'movieId': np.int32,
    'tagId': np.int16,
    'relevance': np.float32
})
genome_tags = pd.read_csv(PATH_TO_DATA + 'genome_tags.csv', index_col='tagId')
link = pd.read_csv(PATH_TO_DATA + 'link.csv', index_col='movieId', dtype={
    'imdbId': np.int32 # так как в tmdb содержатся значения NaN, пока не трогаем
})
movie = pd.read_csv(PATH_TO_DATA + 'movie.csv', index_col='movieId')
rating = pd.read_csv(PATH_TO_DATA + 'rating.csv', parse_dates=['timestamp'], dtype={
    'userId': np.int32,
    'movieId': np.int32,
    'rating': np.float32
})
tag = pd.read_csv(PATH_TO_DATA + 'tag.csv', parse_dates=['timestamp'], dtype={
    'userId': np.int32,
    'movieId': np.int32,
    'tagId': np.int16
})

In [ ]:
# в данной таблице содержится информация
# о релевантности фильма и тега
# (чем меньше значение, тем меньше подходит
# данный тег к фильму)

genome_scores.head()

In [ ]:
# здесь можно посмотреть на значение
# тега по его id

genome_tags.head()

In [ ]:
# как было сказано основной источник данных
# это imdb и tmdb
# поэтому в данной таблице можно увидеть
# соответствие между id фильма в выборке
# и id фильма на imdb и tmdb

# как было отмечено при чтение данных, в колонке tmdbId
# есть значения NaN, заменим их на -1
# так как это категориальная колонка и значения -1 в ней нет
# так же нам может понадобится информация,
# что какого-то фильма нет на сайте tmdb

link['tmdbId'] = link['tmdbId'].fillna(-1).astype(np.int32)
link.head()

In [ ]:
# здесь можно посмотреть информацию о каждом фильме
# его название, год выпуска и жанры (разделены через |)

movie.head()

In [ ]:
# основная информация для нашей задачи,
# по userId можно увидеть какие оценки
# и каким фильмам ставил пользователь
# так же для каждой оценки известно время,
# когда её поставили

rating.head()

In [ ]:
# так же пользователи ставят фильмам
# собственные теги, для которых также 
# известна временная отметка

tag.head()

Как видно, данный набор представляет большие возможности для изучения и задача предсказания оценки лишь одна из возможных.

Модели для предсказания оценок можно поделить на большие категории:
1. Коллаборативная фильтрация $-$ используется информация о прошлом поведение пользователя и информация о других пользователях;
2. Контентная фильтрация $-$ формирует рекомендации на основе ретроспиктивного анализа поведения пользователя, здесь уже может использоваться информация о предметах, информация о пользователях;
3. Гибридные методы $-$ сочетание коллаборативной и контентной фильтрации.

### первичный анализ
Но перед тем как строить модели для начала проведём первичный анализ данных

In [ ]:
rating.shape

In [ ]:
rating['userId'].nunique(), rating['movieId'].nunique()

Как было отмечено в начале, имеется большое количество различных фильмов и пользователей, при этом вторых гораздо больше.

In [ ]:
rating.isna().sum()

In [ ]:
rating['rating'].describe()

Шкала оценок распределена от 0 до 5, но оценку 0 никто не ставит, минимум 0.5.<br>
При это средняя и медианна оценка составили ~3.5

In [ ]:
rating['rating'].unique()

шаг оценки 0.5, поэтому всего 10 различных вариантов

In [ ]:
rating.groupby('userId').agg({
    'rating': [mean, std, median, min, max]
}).describe()

Но если мы будем смотреть на пользователя по отдельности, то получим, что его средние показатели не совпадают с общими. Это наталкивает на мысль, что у каждого пользователя свои идеи об оценках, так для одного средняя оценка 2.5, а для другого 3.5, значит надо анализировать пользователей по отдельности.

In [ ]:
rating.groupby('movieId').agg({
    'rating': [mean, std, median, min, max]
}).describe()

Аналогичная ситуация и для фильмов, есть бестеллеры, которые в целом хорошо оценивают, а есть шлак, которому ставят
плохие оценки.

In [ ]:
genres = set()
for cur_genres in movie['genres'].values:
    genres = genres.union(cur_genres.split('|'))
print(len(genres), list(genres))

В выборке присутствует 19 различных жанров + значение нет жанров, наверняка от жанра зависит средняя оценка по фильму

In [ ]:
ids_genres = [[] for _ in np.arange(20)]
for (id, row) in tqdm_notebook(movie.iterrows(), total=movie.shape[0]):
    for i, genre in enumerate(genres):
        if genre in row[1]:
            ids_genres[i].append(id)
            
for i, id in enumerate(ids_genres):
    print('{} mean is {}'.format(
        list(genres)[i],
        rating.groupby('movieId')['rating'].mean()[id].mean()
    ))

Посчитали среднее по каждому фильму, а потом по каждой категории, так как установили, что для каждого фильма своя средняя оценка<br>
Как видим, для каждой категории также присуще своё среднее значение рейтинга, можно сделать вывод, что этот признак влияет на целевую переменную.

Так же в название фильма указан год, что тоже быть хорошей фичей (данные были взяты с фильмами с 1995 по 2016 год)

In [ ]:
years = list(np.arange(1995, 2017))
ids_years = [[] for _ in np.arange(len(years))]
for (id, row) in tqdm_notebook(movie.iterrows(), total=movie.shape[0]):
    for i, year in enumerate(years):
        if str(year) in row[0]:
            ids_years[i].append(id)
            
for i, id in enumerate(ids_years):
    print('{} mean is {}'.format(
        years[i],
        rating.groupby('movieId')['rating'].mean()[id].mean()
    ))

Здесь колебания среднего уже не такое значительное как у жанров, но всё же такой признак может влиять на целевую переменную.

Вспомним про imdb и tmdb, у некоторых фильмов не было значений tmdb, это значит, что такого фильма нет (не было) в базе tmdb, посмотрим, влияет ли это на оценку:

In [ ]:
data = rating.groupby('movieId')['rating'].mean().reset_index()
data['has_tmdb'] = data['movieId'].apply(lambda x: link.loc[x, 'tmdbId'] != -1)
data.groupby('has_tmdb')['rating'].mean()

Практически не влияет.

### визуальный анализ
первичный анализ показал возможные направления решения, но давайте их подтвердим визуально.

Для начала проверим, что средняя оценка каждого пользователя и фильма имеет большой разброс.

In [ ]:
x = rating.groupby('userId')['rating'].mean().index.values
y = rating.groupby('userId')['rating'].mean().values
sns.regplot(x, y)

In [ ]:
x = rating.groupby('movieId')['rating'].mean().index.values
y = rating.groupby('movieId')['rating'].mean().values
sns.regplot(x, y)

In [ ]:
get_user = lambda id, data: data[data['userId'] == id]['rating']
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(16, 10))
for i in np.arange(9):
    sns.distplot(get_user(i + 1, rating), ax=axes[i // 3, i % 3])
    axes[i // 3, i % 3].set_xlabel('ratings')
    axes[i // 3, i % 3].set_ylabel(i)

Гипотеза подтвердилась, для каждого пользователя действительно своя средняя оценка, которая не приближенна к общей. Для фильмов распределение среднего уже больше похоже на общее среднее, но также имеет большой разброс.

Можно попробовать улучшить положение, если нормализовать оценки, а именно: для каждого пользователя из его оценки вычесть его среднюю оценку. Как изменится средняя оценка тогда:
$$
    \overline{r} = \frac{r_1 + r_2 + \ldots + r_n}{n}
$$$$
    \overline{r'} = \frac{(r_1 - \overline{r}) + \ldots + (r_n - \overline{r})}{n} = \overline{r} - \frac{n \times \overline{r}}{n} = 0
$$
Значит среднее станет 0, в нашей интерпретации это означает, что пользователи имеют одинаковое понимание средней оценки.

In [ ]:
figure(figsize=(30, 7))
sns.barplot(list(genres), [len(i) for i in ids_genres])

In [ ]:
data = pd.DataFrame(rating.groupby('movieId')['rating'].mean()).reset_index()
for genre in tqdm_notebook(genres, total=len(genres)):
    data[genre] = data['movieId'].apply(lambda x: int(genre in movie.loc[x, 'genres']))
data.set_index('movieId', inplace=True)
sns.heatmap(data.corr('spearman'))

Существует корреляция между жанрами и рейтингом, но следует отметить, что и многие жанры между собой коррелируют, хотя значение корреляции мало.

In [ ]:
figure(figsize=(30, 7))
sns.barplot(years, [len(i) for i in ids_years])

In [ ]:
data = pd.DataFrame(rating.groupby('movieId')['rating'].mean()).reset_index()
for year in tqdm_notebook(years, total=len(genres)):
    data[year] = data['movieId'].apply(lambda x: int(str(year) in movie.loc[x, 'title']))
data.set_index('movieId', inplace=True)
sns.heatmap(data.corr('pearson'))

Корреляция крайне мала.

Посмотрим, сколько раз голосовали разные люди:
Так как значений очень много, то разделим на несколько категорий:
1. \>1000;
2. 500-1000;
3. 100-500;
4. 10-100;
5. 1-10;

In [ ]:
def cat(x):
    if x > 1000:
        return 0
    elif x > 500:
        return 1
    elif x > 100:
        return 2
    elif x > 10:
        return 3
    else:
        return 4
count = lambda x: cat(len(x))
    
data = rating.groupby('userId').agg({'rating': count}).astype(int)

figure(figsize=(30, 10))
sns.countplot('rating', data=data)

<b>Вывод:</b> для того, чтобы сравнивать двух пользователей или два фильма необходимо нормализовывать оценки, а именно вычитать из них среднюю оценку пользователя или предмета. Также существуют корреляции с некоторыми дополнительными данными.

### инсайты
1. Были обнаружены пропуски в для некоторых фильмов в колонке tmdb, это связано с тем, что некоторые фильмы есть на imdb, но нет на tmdb. Решение: расставить там -1, так как такого id нет, но мы хотим сохранить информацию о неизвестности + это категориальный признак. Больше пропусков нет, данные хорошие, чистые.
2. Были обнаружены корреляции между рейтингом и жанром фильма, но были опровергнуты между рейтингом и годом выпуска.
3. Данные подготовлены хорошо и аккуратно, большинство польователей совершали примерно равное число голосов.

Найденные факты очень важны, так, к примеру, дополнительные признаки могут улучшить контентные модели, где учитывается информация и ретроспектива прошлых действий. С другой стороны, при решение с помощью коллаборативной фильтрации, очень важно определять близость пользователей или фильмов, поэтому нормализация оценки сделает вычисление схожести приближенной к реальному отношению.

### метрика
Для задач предсказания оценки используются классические метрики регрессии: `MSE`, `RMSE` и `MAE`. Сейчас стандартом оценки качества является `RMSE`, которая приобрела большую популярность после [Netflix Prize](https://ru.wikipedia.org/wiki/Netflix_Prize). Такая метрика выбрана, так как при решение задачи с помощью линейных алгоритмов, например, `SVD`, нам необходимо минизировать функцию потерь, а среднеквадратичная ошибка хорошо для этого подходит.

Как уже было сказано, задача рекомендаций не заканчивается на построение прогнозов оценки, так как дать пользователи фильмы с наибольшей оценкой не всегда верно, именно поэтому иногда используют метрики ранжирования, например, `nDCG`, но так как мы ставим себе задачу именно предсказания оценки, то данный вариант не подходит.

Так же, в случаях двух возможных оценок, например, лайк и дизлайк, получаем задачу классификации, где используются уже стандартные метрики классификации: `Precision` и `Recall`. Но такой вариант также не подходит.

Таким образом, для оценки качества алгоритма будем использовать метрику `RMSE`. Её нет в пакете `sklearn`, но её легко получить с помощью встроенной `MSE`:

In [ ]:
from surprise.accuracy import rmse

### выбор модели
Для решения данной задачи воспользуемся готовыми решениями библиотеки [Surprise](surprise.readthedocs.io/en/stable/index.html). Это `scikit` пакет, разработанный специально для рекомендательных систем, в нём есть реализации наиболее популярных алгоритмов: `KNN`, `SVD` и `CoClustering`, а так же предоставленны удобные функции для валидации и подбора параметров. Я воспользуюсь `SVD` алгоритмом, он получил свою известность, благодаря `Netflix Prize`, именно эту модель использовали лидеры соревнований. Сейчас это один из основных применяемых алгоритмов при постраение коллаборативных моделей, он куда более продуманный, чем `user-based` и `item-based` модели. Основным отличием этой модели является возможность выявлять скрытые признаки объектов и интересы пользователей. Например, может так получиться, что на первой координате вектора у каждого пользователя будет стоять число, показывающее, похож ли пользователь больше на мальчика или на девочку, на второй координате — число, отражающее примерный возраст пользователя. У фильма же первая координата будет показывать, интересен ли он больше мальчикам или девочкам, а вторая — какой возрастной группе пользователей он интересен. 

### предобработка данных
Но для начала подготовим данные:
- Так как делается упор на коллаборативную фильтрацию, то необходимы номера пользователей и фильмов, а также какие рейтинги были проставлены.
- Нормализуем рейтинги путём вычитания из них средней оценки пользователя.
- Разобьём данные на тренировочную и тестовую часть, просто отрезать часть выборки будет неправильно, так как алгоритм не должен знать о будующих оценках, так же неправильно убирать пользователя целиком из данных, так как для него тогда не будет похожих пользователей (проблема холодного старта для моделей рекомендательных систем). Поступим так: для каждого пользователя отрежем последние $30%$ оценок, для этого воспользуемся колонкой `timestamp`.
- Интерфейс `Surprise` не предпологает выделения целевой переменной, а требует передать данные с $3$ колонками в порядке: $\textit{user id, item id, ratings}$

In [ ]:
data = rating.copy()

In [ ]:
def train_test_split(X, train_size=0.7, user_col='userId', item_col='movieId',
                     rating_col='rating', time_col='timestamp'):
    X.sort_values(by=[time_col], inplace=True)
    user_ids = X[user_col].unique()
    X_train_data = []
    X_test_data = []
    for user_id in tqdm_notebook(user_ids):
        cur_user = X[X[user_col] == user_id]
        idx = int(cur_user.shape[0] * train_size)
        X_train_data.append(cur_user[[user_col, item_col, rating_col]].iloc[:idx, :].values)
        X_test_data.append(cur_user[[user_col, item_col, rating_col]].iloc[idx:, :].values)
    X_train = pd.DataFrame(np.vstack(X_train_data), columns=[user_col, item_col, rating_col])
    X_test = pd.DataFrame(np.vstack(X_test_data), columns=[user_col, item_col, rating_col])
    return X_train, X_test

# аккуратно, очень долгий процесс
X_train, X_test = train_test_split(data)
X_train.shape, X_test.shape

In [ ]:
# разделив 1 раз, сохраним учебную и тренировочную часть в файлы

# X_train.to_csv(PATH_TO_DATA + 'train.csv')
# X_test.to_csv(PATH_TO_DATA + 'test.csv')

X_train = pd.read_csv(PATH_TO_DATA + 'train.csv', index_col='Unnamed: 0',
                      dtype={'userId': np.int32, 'movieId': np.int32, 'rating': np.float16})
X_test = pd.read_csv(PATH_TO_DATA + 'test.csv', index_col='Unnamed: 0',
                      dtype={'userId': np.int32, 'movieId': np.int32, 'rating': np.float16})

In [ ]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(-5.0, 5.0)) # после нормализации шкала оценок стало от -5.0 до +5.0
X_train_surp = Dataset.load_from_df(X_train, reader) # специальный формат данных, поддерживающий деление на фолды
X_test_surp = Dataset.load_from_df(X_test, reader)

### Кросс-валидация и подбор параметров
`SVD` $-$ линейная модель, поэтому для неё характерны стандартные параметры этого семейства алгоритмов:
1. Коэффициент регуляризации;
2. Колличество шагов градиентного спуска;
3. Размер шага градиентного спуска.

Ознакомится со всеми параметрами данной модели можно [здесь](http://surprise.readthedocs.io/en/stable/matrix_factorization.html), стоит заметить, что здесь учитывается разброс данных, который был доказан нами ранее, поэтому в модели указывается `biased=True`.

Подбор гиперпараметров будет осуществляться с помощью [`GridSearch`](http://surprise.readthedocs.io/en/stable/model_selection.html#surprise.model_selection.search.GridSearchCV), а валидироваться с помощью кросс-валидации по принципу [`LeaveOneOut`](http://surprise.readthedocs.io/en/stable/model_selection.html#surprise.model_selection.split.LeaveOneOut) $-$ у каждого пользователя $1$ рейтинг попадает в тестовый.

Мои вычислительные мощности не позволяют обучить так много `SVD` моделей, поэтому я воспользуюсь стандартными настройками, проверю модель на отложенной выборке, а затем сравню с результатом на тестовых данных.

In [ ]:
%%time
from surprise import SVD
from surprise.model_selection import LeaveOneOut


fold = LeaveOneOut(n_splits=1, random_state=7)
model = SVD(n_epochs=10, biased=True, random_state=7, verbose=True)

for trainset, valset in fold.split(X_train_surp):
    model.fit(trainset)
    rmse(model.test(valset))

### Создание новых признаков
Так как разрабатывалсь коллаборативная модель, то никаких дополнительных признаков <b>не используется</b>, в этом её суть. Однако существуют мощные модели контентной фильтрации, где информация о пользователе и предмете активно используется, один из самых распространённых алгоритмов в этом направление $-$ факторизационные машины.

Для таких моделей из наших данных можно вытащить множество признаков, например, как уже было показано $-$ жанр фильма. Другой возможный признак: нам известно какой тег пользователь поставил фильму, также нам известна релевантность тега к фильму. Если релевантность низкая, то, возможно, пользователь не до конца понял фильм и оценка некорректна, таким образом можно штрафовать рейтинг юзера, например:
$$
    r_{ui}' = \overline{r_u} + \textit{rel(u, i)} \times (r_{ui} - \overline{r_u})
$$
То есть при низкой релевантности приближаем оценку пользователя к его средней оценке. Ни в коем случае нельзя удалять записи с низкой релевантностью или по каким-либо другим идейным соображениям, так как данный пользователь смотрел данный фильм, а эту информацию нельзя терять иначе будут создаваться рекомендации для объектов, которые уже посещались.

### Прогноз на тестовой или отложенной выборке
Так как данные были взяты не из соревнования, то нет возможности смотреть скор на LB, для этого проверяем модель на отложенной выборке, построение которой было описано ранее: для каждого пользователя $30%$ последних оценок попадают в тестовую выборку. Это очень похоже на разбиение при кросс-валидации, но, что логично, берём уже несколько оценко пользователя. 

In [ ]:
%%time
model = SVD(n_epochs=10, biased=True, random_state=7, verbose=True)
model.fit(X_train_surp.build_full_trainset())

In [ ]:
predictions = model.test(X_test_surp.build_full_trainset().build_testset())
rmse(predictions)

### оценка качества
получили хорошую оценку, которая близка к оценке на отложенной выборке, что говорит об устойчивости модели. Так как не было кросс-валидации, то и кривые строит не на чем...

### выводы
Построение рекомендательных систем важная бизнес-задача, чем дольше пользователь остаётся на сайте, тем больше денег он приносит. В данном проекте был рассмотрен процесс постороения коллаборативной системы, конечно её ещё есть куда улучшить, помимо `SVD` существует множество других алгоритмов. Если продолжать работу именно с этим алгоритмом, то можно двигаться в сторону улучшения модели путём перебора параметров по сетке.

Но тем не менее, основная ценность данного решения $-$ существует модель, которая может предсказать будущие оценки пользователя, а так как это линейная модель, то достаточно иметь веса, чтобы их делать. Таким образом мы получили легко интегрируемую функцию, что очень хорошо вписывается в web-сервисы. Также данные модели можно дообучать, что несомненно ещё один плюс в наше решение.